In [1]:
# LangChain과 YOLO를 활용한 이미지 감지 및 LLM 기반 설명 생성 코드
!pip install -U langchain langchain-core langchain-community langchain-google-genai google-genai langchain-openai python-dotenv
!pip install -U sentence-transformers # 문장 임베딩 관련 라이브러리 (코드 내에서는 직접 사용되지 않으나, LangChain/RAG 환경에서 일반적)
!pip install -U chromadb # 벡터 데이터베이스 관련 라이브러리 (코드 내에서는 직접 사용되지 않으나, LangChain/RAG 환경에서 일반적)
!pip install ultralytics # YOLO 모델을 사용하기 위한 라이브러리

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.5 MB/s e

In [2]:
import os, io, base64
from pathlib import Path
from dotenv import load_dotenv
from typing import List, Dict # 타입 힌트 (가독성 및 코드 안정성 향상)
from PIL import Image         # Pillow 라이브러리: 이미지 파일을 열고 조작
from ultralytics import YOLO
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

# LLM (Large Language Model) 설정
llm = ChatGoogleGenerativeAI(
    temperature=0.4,  # 모델의 창의성/무작위성 조절 (낮을수록 일관된 답변)
    model="gemini-2.5-flash", # 이미지 분석에 최적화된 빠른 멀티모달 모델 사용
    google_api_key=userdata.get('GOOGLE_API_KEY') # Colab의 보안 저장소에서 API 키를 가져와 설정
)

# YOLO (You Only Look Once) 모델 로드
# yolov10n.pt: YOLOv10의 '나노(nano)' 버전, 빠르고 가벼운 모델 가중치 파일
yolo_model = YOLO("yolov10n.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [21]:
# image -> base64 반환
# PIL 이미지 객체를 LLM이 인식할 수 있는 Data URL 형식의 Base64 문자열로 변환
def pil_to_data_urlFunc(img:Image.Image, format:str="JPEG") -> str:
  buf = io.BytesIO()  # 메모리 기반의 바이트 버퍼 생성
  img.save(buf, format=format)  # PIL 이미지를 지정포맷으로 메모리 버퍼에 저장
  img_bytes = buf.getvalue()    # 버퍼에 저장된 바이트 데이터 가져오기
  b64 = base64.b64encode(img_bytes).decode("utf-8") # 바이트를 Base64로 인코딩 후 UTF-8 문자열로 디코딩
  return f"data:image/jpeg;base64, {b64}" #LLM이 이해할 수 있는 data URL 형태의 문자열 반환, Data URL 형식 (MIME 타입 + Base64 데이터) 반환


# 음식 감지 (YOLOv10 사용)
def detect_dishesFunc(image_path:str, conf_thres:float=0.3) -> List[Dict]:
  # YOLO 모델 실행. [0]은 첫 번째 이미지의 감지 결과 객체를 가져옴
  results = yolo_model(image_path)[0]
  image = Image.open(image_path).convert("RGB") # 원본 이미지 로드
  dishes = []
  boxes = results.boxes # 감지된 객체의 바운딩 박스 정보
  if boxes is None:
    return dishes      # 감지된 객체가 없으면 빈 리스트 반환

  names = results.names   # YOLO 모델이 학습된 클래스 이름 목록
  # boxes.xyxy: 바운딩 박스 좌표 (x1, y1, x2, y2)
  # boxes.cls: 감지된 객체의 클래스 ID
  # boxes.conf: 감지된 객체의 신뢰도(Confidence)
  # print("names:", names) # Debugging line, can be removed if not needed
  for box, cls, conf in zip(boxes.xyxy, boxes.cls, boxes.conf):
    if conf < conf_thres:   #신뢰도가 임계값보다 낮으면 무시
      continue

    x1, y1, x2, y2 = map(int, box.tolist()) # 좌표를 정수형으로 변환
    crop = image.crop((x1, y1, x2, y2)) # 바운딩 박스 영역만큼 원본 이미지에서 잘라내기 (크롭)
    label = names[int(cls)]             # 클래스 ID에 해당하는 라벨(이름) 가져오기
    print("label:", label)
    dishes.append({
        "label": label,     # 감지된 객체 라벨 (예: 'pizza', 'apple')
        "crop": crop,       # 잘라낸 PIL 이미지 객체
        "conf": float(conf), # 신뢰도
        "bbox": [x1, y1, x2, y2]  #바운딩 박스 좌표
    })

  # 신뢰도(conf)를 기준으로 내림차순 정렬 (가장 확실한 감지 결과를 먼저 처리)
  # conf 별 내리차순
  dishes.sort(key=lambda d:d["conf"], reverse=True)
  return dishes


# llm에게 레시피 요청
def ask_recipe_with_llmFunc(dish_img:Image.Image, label:str | None = None) -> str:
  data_url = pil_to_data_urlFunc(dish_img)    # PIL 이미지를 Data URL로 변환
  base_prompt =  (
      "너는 전문 요리사이자 레시피 개발자야\n"
      "다음 음식 사진을 보고 아래 내용을 한국어로 자세히 설명해줘\n"
      "1. 이 요리의 이름(추정)과 특징을 간단히 설명\n"
      "2. 사용할 재료 목록 설명 \n"
      "3. 조리 순서 단계별로을 친절하게 설명\n"
      "4. 특별히 맛있게 만드는 법 설명\n"
      "5. 이 요리와 비슷한 다른 요리 3개 추천해 줘\n"
      "중요사항 : 마크다운 문법(**,*,-)등을 사용하지말고 순수한 평문으로만 답을 해줘"

  )
  if label:
    # YOLO 감지 결과(label)를 프롬프트에 추가하여 LLM의 답변 정확도 향상 시도 (참조)
    base_prompt += f"\n 참고 : YOLO 모델이 이 음식을 '{label}'로 감지함. 얘를 최대한 활용"
  # LangChain의 HumanMessage 구조를 사용하여 멀티모달 입력 구성
  msg = HumanMessage(
      content = [
          {"type":"text", "text":base_prompt},  # 텍스트 프롬프트
          {"type":"image_url", "image_url":data_url}, # 이미지 데이터 URL
      ]
  )
  resp = llm.invoke([msg])  # LLM 호출 및 응답 받기
  return resp.content       # 응답 내용(텍스트) 반환

# 전체 파이프 라인
def process_food_imageFunc(image_path:str,max_dishes:int=2) -> None:
  print(f"입력 이미지 : {image_path}")
  dishes = detect_dishesFunc(image_path)  # YOLO로 이미지 내의 음식 객체 감지
  if not dishes:
    print("음식 이미지가 없어요. 전체 이미지를 그대로 LLM에게 전달합니다.")
    whole_img = Image.open(image_path).convert("RGB")
    answer = ask_recipe_with_llmFunc(whole_img, label=None)
    print("\n LLM응답 (전체 이미지) == \n", answer)
    return
  print(f"감지된 음식 수 : {len(dishes)}")

  # 감지된 리스트 중 앞에서부터 max_dishes개 만큼 잘라서 사용
  # 감지된 음식 중 신뢰도 순으로 max_dishes개만 처리
  for i, dish in enumerate(dishes[:max_dishes]):
    label = dish["label"]
    crop = dish["crop"]
    bbox = dish["bbox"]
    print(f"\n[{i}] 감지된 음식---")
    print(f"라벨:{label}, 신뢰도:{dish["conf"]:.3f}, bbox:{bbox}")
    # 감지된 음식 이미지 레시피 추천 요청 / 크롭된 이미지와 라벨을 LLM에 전달하여 레시피 요청
    answer = ask_recipe_with_llmFunc(dish["crop"], label=label)
    print(f"\nLLM응답(라벨:{label}) ==\n ", answer)



process_food_imageFunc("food.jpg",max_dishes=1)

입력 이미지 : food.jpg

image 1/1 /content/food.jpg: 512x640 1 pizza, 176.7ms
Speed: 5.1ms preprocess, 176.7ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)
label: pizza
감지된 음식 수 : 1

[0] 감지된 음식---
라벨:pizza, 신뢰도:0.933, bbox:[7, 7, 641, 512]

LLM응답(라벨:pizza) ==
  안녕하세요, 전문 요리사이자 레시피 개발자입니다. 보여주신 사진 속 음식은 보기만 해도 군침이 도는 맛있는 피자네요. 제가 이 피자에 대해 자세히 설명해 드릴게요.

1.  이 요리의 이름(추정)과 특징을 간단히 설명
    사진 속 요리는 다양한 토핑이 어우러진 콤비네이션 피자로 추정됩니다. 특히 짭짤한 블랙 올리브와 달콤한 스위트 콘, 그리고 아삭한 초록 피망이 듬뿍 올라가 있어 다채로운 색감과 풍부한 식감을 자랑합니다. 고소하게 녹아내린 모짜렐라 치즈가 이 모든 재료를 부드럽게 감싸 안아 남녀노소 누구나 좋아할 만한 대중적인 맛이 특징입니다. 쫄깃한 도우와 바삭한 엣지, 그리고 신선한 토핑의 조화가 일품인 피자입니다.

2.  사용할 재료 목록 설명
    이 맛있는 피자를 만들기 위해 필요한 재료들은 다음과 같습니다.

    피자 도우 재료:
    강력분 200g
    드라이 이스트 4g
    설탕 5g
    소금 3g
    올리브 오일 10ml
    따뜻한 물 120ml

    피자 소스 재료:
    시판 토마토 피자 소스 100g
    오레가노 가루 약간
    바질 가루 약간
    (선택 사항) 다진 마늘 1/2 작은술

    토핑 재료:
    모짜렐라 치즈 200g (넉넉하게)
    블랙 올리브 슬라이스 30g
    스위트 콘 (통조림) 30g
    초록 피망 1/2개 (잘게 다진 것)
    (선택 사항) 양파 1/4개 (잘게 다

단계/기술/역할/데이터 형태

1단계/YOLOv10(객체감지)/이미지에서 특정 객체(음식)의 위치(bbox)와 종류(label)감지하고 잘라냄/이미지파일->크롭된 이미지(PIL), 바운딩 박스 좌표, 라벨


2단계/Base64 인코딩/ 크롭된 이미지를 텍스트(data URL)형태로 변환하여 LLM에게 전달할 수 있도록 준비/크롭된 이미지(PIL -> BASE64문자열)


3단계/LangChain/base64인코딩된 이미지와 텍스트 프롬프트를 받아 복합적으로 이해하고 요청한 레시피 설명을 생성/ Base64+텍스트 프롬프트 -> 텍스트 응답


객체 감지 모델 : YOLO (You Only Look Once)
실시간 객체 감지(Object Detection)를 위해 개발된 알고리즘 계열. 다른 방식들이 이미지를 여러 번 분석하는 데 비해, YOLO는 단 한 번의 신경망 통과로 객체의 위치와 클래스를 동시에 예측하여 속도가 매우 빠름
yolov10n.pt: YOLOv10 모델 중 'n' (nano) 버전으로, 가장 가볍고 빠르지만 정확도는 다른 큰 모델(s, m, l 등)보다 낮을 수 있음.(실습이나 모바일 환경에 적합).
신뢰도 (conf): 모델이 해당 객체를 특정 클래스로 예측할 때 얼마나 확신하는지를 나타내는 값.
0에서 1 사이의 값을 가지며, conf_thres (0.3)를 사용하여 낮은 신뢰도의 감지 결과는 무시하고 있음 -> 이는 False Positive (오탐지)를 줄이는 데 중요.
